# Model merging

https://huggingface.co/docs/peft/developer_guides/model_merging

https://towardsdatascience.com/merge-large-language-models-with-mergekit-2118fb392b54

# 1 - Introduction

Training a model for each task can be costly, take up storage space, and the models aren't able to learn new information to improve their performance. Multitask can overcome some of these limitations by training a model to learn several tasls, but it is expensive to train and designing a dataset for it is challenging.

*Model merging* offers a solution to these challenges by combining multiple pretrained models into one model, giving it the combined abilities of each individual model without any additional training.

# 2 - Merge algorithms

## 2.1 - Linear

The classic merge method - a simple weighted average

## 2.2 - SLERP

Spherical Linear Interpolation (SLERP) is a method used to smoothly interpolate between two vectors. It maintains a constant rate of change and preserves the geometric properties of the spherical space in which the vectors reside.

There are several reasons to prefer SLERP over a traditional linear interpolation. For example, in high-dimensional spaces, linear interpolation can lead to a decrease in the magnitude of the interpolated vector (i.e., it reduces the scale of weights). Moreover, the change in direction of the weights often represents more meaningful information (like feature learning and representation) than the magnitude of change.

SLERP is implemented using the following steps:

1. Normalize the input vectors to unit length, ensuring they represent directions rather than magnitudes

2. Calculate the angle between these vectors using their dot product

3. If the vector are nearly collinear (i.e., parallel), it defaults to linear interpolation for efficiency. Otherwise, SLERP computes scale factors based on the interpolation factor `t` (`t=0` -> 100% of the first vector, `t=1` -> 100% of the second vector) and the angle between the vectors.

4. These factors are used to weigh the original vectors, which are then summed to obtain the interpolated vector

SLERP is currently the most popular merging method, but **it is limited to combining only two models at a time**. It is still possible to hierarchically combine multiple models, as shown in [Mistral-7B-Merge-14-v0.1](https://huggingface.co/EmbeddedLLM/Mistral-7B-Merge-14-v0.1).

## 2.3 - TIES

Introduced in [this paper by Yadav et al. (2023)](https://arxiv.org/abs/2306.01708), TIES-Merging is designed to efficiently merge multiple task-specific models into a single multitask model. It addresses two main challenges in model merging:

* **Redundancy in model parameters:** It identifies and eliminates redundant parameters within task-specific models. This is achieved by focusing on the changes made during fine-tuning, identifying the top-k% mos significant changes, and discarding the rest.

* **Disagreement between parameter signs:** Conflicts arise when different models suggest opposing adjustments to the same parameter. TIES-Merging resolves these conflicts by creating a unified sign vector that represents the most dominant direction of change across al models.

TIES-Merging is divided into the following three steps:

1. **Trim:** Reduces redundancy in task-specific models by retaining only a fraction the most significant parameters (density parameter) and resetting the rest to zero.

2. **Elect Sign:** Resolves sign conflicts across different models by creating a unified sign vector based on the most dominant direction (positive or negative) in terms of cumulative magnitude.

3. **Disjoint Merge:** Averages parameter values that align with the unified sign vector, excluding zero values.

Unlike SLERP, **TIES can merge multiple models at a time**.

## 2.4 - DARE

Drop And REscale is a method that can be used to prepare for other model merging methods like TIES. it works by randomly dropping parameters according to a drop rate and rescaling the remaining parameters. This helps to reduce the number of redundant and potentially interfering parameters among multiple models.

Introduced by [Yu et al. (2023)](https://arxiv.org/abs/2311.03099), DARE uses an approach similar to TIES with two main differences:

* **Pruning:** DARE randomly reset fine-tuned weights to their original values (those of the base model).

* **Rescaling:** DARE rescales the weights to keep the expectations of the model outputs approximately unchanged. It adds the rescaled weights of both (or more) models to the weights of the base model with a scale factor.

## 2.5 - Passthrough

The passthrough method differs significantly from the previous ones. By concatenating layers from different LLMs, it can produce models with an [exotic number of parameters](https://arxiv.org/abs/2311.03099) (e.g., 9B from two 7B parameter models). These models are often referred as "frankenmerges" or "Frankenstein models" by the community.

This technique is very experimental, but it managed to create impressive models, like [goliath-120b](https://huggingface.co/alpindale/goliath-120b) using two Llama 2 70B models. The recently released [SOLAR-10.7B-v1.0](https://huggingface.co/upstage/SOLAR-10.7B-v1.0) also uses the same idea, [called depth-up scaling in their paper](https://arxiv.org/abs/2312.15166).

# 3 - Model merging using HuggingFace PEFT

Let's show how can we easily merge three models using TIES and DARE in HuggingFace PEFT. Models are merged with the `add_weighted_adapter()` method, and the specific model merging method is specified in the `combination_type` parameter.

For example, let's merge three finetuned [TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T](https://huggingface.co/TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T) models: 
* [tinyllama_lora_nobots](https://huggingface.co/smangrul/tinyllama_lora_norobots)
* [tinyllama_lora_sql](https://huggingface.co/smangrul/tinyllama_lora_sql)
* [tinyllama_lora_adcopy](https://huggingface.co/smangrul/tinyllama_lora_adcopy)

-----

**Note:**

When you are attempting to merge fully trained models with TIES, you should be aware of any special tokens each model may have added to the embedding layer which are not a part of the original checkpoint's vocabulary. This may cause an issue because each model may have added a special token to the same embedding position. If this is the case, you should use the [`resize_token_embeddings()` method](https://huggingface.co/docs/transformers/v4.39.0/en/main_classes/model#transformers.PreTrainedModel.resize_token_embeddings) to avoid merging special tokens at the same embedding index.

**This shouldn't be an issue if you are only merging LoRA adapters trained from the same base model** (as is the case in this example).

----

Load a base model anduse the `load_adapter()` method to load and assign each adapter a name:

In [9]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

config = PeftConfig.from_pretrained("smangrul/tinyllama_lora_norobots")
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, load_in_4bit=True, device_map="auto").eval()
tokenizer = AutoTokenizer.from_pretrained("smangrul/tinyllama_lora_norobots")

model.resize_token_embeddings(len(tokenizer)) # 32000 -> 32005

model = PeftModel.from_pretrained(model, "smangrul/tinyllama_lora_norobots", adapter_name="norobots")
_ = model.load_adapter("smangrul/tinyllama_lora_sql", adapter_name="sql")
_ = model.load_adapter("smangrul/tinyllama_lora_adcopy", adapter_name="adcopy")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 3.1 - TIES

Weight values greater than `1.0` typically produce better results because they preserve the correct scale. A good default starting value for the weights is to set all values to `1.0`.

In [10]:
adapters = ["norobots", "adcopy", "sql"]
weights = [2.0, 1.0, 1.0]
adapter_name = "merge"
density = 0.2
model.add_weighted_adapter(adapters, weights, adapter_name, combination_type="ties", density=density)

Set the newly merged models as the active model with the `set_adapter()` method.

In [11]:
model.set_adapter("merge")

Now you can use the merged model as an instruction-tuned model to write ad copy or SQL queries!

### 3.1.1 - Instruct

In [12]:
messages = [
    {"role": "user", "content": "Write an essay about Generative AI."},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")
inputs = {k: v.to("cuda") for k, v in inputs.items()}
outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, top_p=0.95, temperature=0.2, repetition_penalty=1.2, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

/anaconda/envs/pytorch/lib/python3.9/site-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


<s><|im_start|>user 
Write an essay about Generative AI.<|im_end|> 
<|im_start|>assistant 
Write a paper on the topic of your choice.<|im_end|>


### 3.1.2 - ad copy

In [13]:
messages = [
    {"role": "system", "content": "Create a text ad given the following product and description."},
    {"role": "user", "content": "Product: Sony PS5 PlayStation Console\nDescription: The PS5 console unleashes new gaming possibilities that you never anticipated."},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")
inputs = {k: v.to("cuda") for k, v in inputs.items()}
outputs = model.generate(**inputs, max_new_tokens=128, do_sample=True, top_p=0.95, temperature=0.2, repetition_penalty=1.2, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

<s><|im_start|>system 
Create a text ad given the following product and description.<|im_end|> 
<|im_start|>user 
Product: Sony PS5 PlayStation Console
Description: The PS5 console unleashes new gaming possibilities that you never anticipated.<|im_end|> 
<|im_start|>assistant 
Ad: Experience the next generation of gaming with the Sony PS5! 🎮🌟 Unlock endless possibilities for your favorite gamesational titles. Your gaming experience will be enhanced by cutting-edge technology, featuring an immersive soundtrack, and more. 🔊🧘‍♂️👾🚀🕹🍿🏞️🖥️📱🎯🦈💃🪄🐶🗺


### 3.1.3 - SQL

In [14]:
text = """Table: 2-11365528-2
Columns: ['Team', 'Head Coach', 'President', 'Home Ground', 'Location']
Natural Query: Who is the Head Coach of the team whose President is Mario Volarevic?
SQL Query:"""

inputs = tokenizer(text, return_tensors="pt")
inputs = {k: v.to("cuda") for k, v in inputs.items()}
outputs = model.generate(**inputs, max_new_tokens=64, repetition_penalty=1.1, eos_token_id=tokenizer("</s>").input_ids[-1])
print(tokenizer.decode(outputs[0]))

<s> Table: 2-11365528-2
Columns: ['Team', 'Head Coach', 'President', 'Home Ground', 'Location']
Natural Query: Who is the Head Coach of the team whose President is Mario Volarevic?
SQL Query: SELECT Head Coach FROM Team WHERE President = Mario Volarevic</s>


## 3.2 - DARE

In [15]:
adapters = ["norobots", "adcopy", "sql"]
weights = [2.0, 0.3, 0.7]
adapter_name = "merge"
density = 0.2
model.add_weighted_adapter(adapters, weights, adapter_name, combination_type="dare_ties", density=density)

### 3.2.1 - Instruct

In [16]:
messages = [
    {"role": "user", "content": "Write an essay about Generative AI."},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")
inputs = {k: v.to("cuda") for k, v in inputs.items()}
outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, top_p=0.95, temperature=0.2, repetition_penalty=1.2, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

<s><|im_start|>user 
Write an essay about Generative AI.<|im_end|> 
<|im_start|>assistant 
Generate a new sentence using the following rules:
1. The is/subject must be in the present tense of the verb
2. The is/verb must be in the past tense of the subject
3. Write an essay about Generative AI.<|im_end|>


### 3.2.2 - ad copy

In [17]:
messages = [
    {"role": "system", "content": "Create a text ad given the following product and description."},
    {"role": "user", "content": "Product: Sony PS5 PlayStation Console\nDescription: The PS5 console unleashes new gaming possibilities that you never anticipated."},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")
inputs = {k: v.to("cuda") for k, v in inputs.items()}
outputs = model.generate(**inputs, max_new_tokens=128, do_sample=True, top_p=0.95, temperature=0.2, repetition_penalty=1.2, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

<s><|im_start|>system 
Create a text ad given the following product and description.<|im_end|> 
<|im_start|>user 
Product: Sony PS5 PlayStation Console
Description: The PS5 console unleashes new gaming possibilities that you never anticipated.<|im_end|> 
<|im_start|>assistant 
Ad: Experience the future of gaming with the Sony PS5! 🎮🌟 Unlock endless possibilities for your favorite gamesational titles. #PlayNow #PS5 #GamersLife #SonyPlaystationConsole 🔍👾🚀🏁️🧘‍♂️🕹🦸💻📱🖥️�����������������������������


### 3.2.3 - SQL

In [18]:
text = """Table: 2-11365528-2
Columns: ['Team', 'Head Coach', 'President', 'Home Ground', 'Location']
Natural Query: Who is the Head Coach of the team whose President is Mario Volarevic?
SQL Query:"""

inputs = tokenizer(text, return_tensors="pt")
inputs = {k: v.to("cuda") for k, v in inputs.items()}
outputs = model.generate(**inputs, max_new_tokens=64, repetition_penalty=1.1, eos_token_id=tokenizer("</s>").input_ids[-1])
print(tokenizer.decode(outputs[0]))

<s> Table: 2-11365528-2
Columns: ['Team', 'Head Coach', 'President', 'Home Ground', 'Location']
Natural Query: Who is the Head Coach of the team whose President is Mario Volarevic?
SQL Query: SELECT Head Coach FROM Team WHERE President = Mario Volarevic</s>
